In [1]:
import numpy as np
import pandas as pd

train = pd.read_csv("./Traffic_accident_info/Traffic_accident_info(12.1 ~ 17.6).csv", encoding='ms949')
day_weather = pd.read_csv("./subdata/11. Observe daily weather in several district/observe daily weather(2012~2017).csv", encoding='ms949')

# '발생년', '사고유형', '법규위반_대분류', '도로형태', '당사자종별_1당', '당사자종별_2당'
for idx in [22,20,15,14, 0]:
    train = train.drop(columns=train.columns[idx])
    
train_col = train.columns
day_weather_col = day_weather.columns

In [2]:
# Get 발생지시도 - 서울 dataset 
train_array = np.array(train)
seoul_dataset = []

for i in range(len(train_array)):
    if train_array[i,9] == '서울':
        seoul_dataset.append(train_array[i])
        
seoul_dataset = np.array(seoul_dataset)

day_order = seoul_dataset[:,0]
sorted_order = sorted(range(len(day_order)), key=lambda k: day_order[k])
seoul_dataset=seoul_dataset[[sorted_order]]

# check result
print(len(seoul_dataset),list(train['발생지시도']).count('서울'))

2050 2050


In [3]:
# make day_weather DataFrame to dict
uniq_days = list(map(str,set(day_weather[day_weather.columns[0]])))
uniq_regions = list(set(day_weather[day_weather.columns[1]]))

day_weather_dict = dict.fromkeys(sorted(uniq_days))

for day in uniq_days:
    day_weather_dict[day] = dict.fromkeys(sorted(uniq_regions))
    for rg in uniq_regions:
        day_weather_dict[day][rg] = list()

for day in np.array(day_weather):
    day_weather_dict[str(day[0])][day[1]].append(day[2:])

In [4]:
# Concat Seoul Traffic Dataset with Seoul Day Weather
seoul_dataset = list(seoul_dataset)
error_indices = []

# try concat but except when missing data or key values error has been raised"
for i in range(len(seoul_dataset)):
    rg = '중구' if seoul_dataset[i][10] == '중구' else seoul_dataset[i][10][:-1]
    try :
        if not day_weather_dict[str(seoul_dataset[i][0])[:8]][rg] :
            error_indices.append(i)
        else :
            seoul_dataset[i] = np.concatenate((seoul_dataset[i],
                                               day_weather_dict[str(seoul_dataset[i][0])[:8]][rg][0]))
    except : 
        error_indices.append(i)

# Remove Error rows by missing data
for i in list(reversed(error_indices)):
    seoul_dataset.pop(i)
    
seoul_dataset = np.array(seoul_dataset)

In [5]:
# Save processed Dataset as csv file without one-hot encoding
seoul_col = train_col.append(day_weather_col[2:])
seoul_dataset_pd = pd.DataFrame(seoul_dataset, columns=seoul_col)

In [6]:
# get columns' names that have string values For one-hot encoding
str_col = []
for i, w in enumerate(seoul_dataset[0]):
    if type(w) is str:
        str_col.append(i)

In [7]:

# get unique target class name# get u 
uniq_class = []

for idx in str_col:
    uniq_class.append(list(sorted(set(seoul_dataset[:,idx]))))

In [9]:
# One Hot Encoding
for idx in range(len(seoul_dataset)):
    for i, col in enumerate(str_col):
        seoul_dataset[idx,col] = int(uniq_class[i].index(seoul_dataset[idx,col]))
        
# Save processed Dataset as csv file with one-hot encoding
seoul_dataset_pd = pd.DataFrame(seoul_dataset, columns=seoul_col)

In [10]:
uniq_class.pop(3)

['강남구',
 '강동구',
 '강북구',
 '강서구',
 '관악구',
 '광진구',
 '구로구',
 '금천구',
 '노원구',
 '도봉구',
 '동대문구',
 '동작구',
 '마포구',
 '서대문구',
 '서초구',
 '성동구',
 '성북구',
 '송파구',
 '양천구',
 '영등포구',
 '용산구',
 '은평구',
 '종로구',
 '중구',
 '중랑구']

In [11]:
uniq_class.pop(2)

['서울']

In [12]:
uniq_class

[['야간', '주간'],
 ['금', '목', '수', '월', '일', '토', '화'],
 ['차대사람', '차대차', '차량단독'],
 ['공작물충돌',
  '기타',
  '길가장자리구역통행중',
  '도로이탈',
  '보도통행중',
  '전도',
  '전도전복',
  '정면충돌',
  '주/정차차량 충돌',
  '차도통행중',
  '추돌',
  '측면직각충돌',
  '측면충돌',
  '횡단중'],
 ['과로',
  '과속',
  '교차로 통행방법 위반',
  '기타(운전자법규위반)',
  '보행자 보호의무 위반',
  '부당한 회전',
  '신호위반',
  '안전거리 미확보',
  '안전운전 의무 불이행',
  '앞지르기 방법위반',
  '중앙선 침범',
  '직진 및 우회전차의 통행방해',
  '차로위반(진로변경 위반)'],
 ['고가도로위', '교차로', '기타', '기타/불명', '단일로', '지하도로내'],
 ['고가도로위',
  '교량위',
  '교차로내',
  '교차로부근',
  '교차로횡단보도내',
  '기타',
  '기타/불명',
  '기타단일로',
  '지하도로내',
  '지하차도(도로)내',
  '터널안',
  '횡단보도부근',
  '횡단보도상'],
 ['건설기계', '불명', '승용차', '승합차', '원동기장치자전거', '이륜차', '자전거', '특수차', '화물차'],
 ['건설기계',
  '보행자',
  '불명',
  '승용차',
  '승합차',
  '없음',
  '원동기장치자전거',
  '이륜차',
  '자전거',
  '특수차',
  '화물차'],
 ['-0.1',
  '-0.100000001490116',
  '-0.2',
  '-0.200000002980232',
  '-0.3',
  '-0.300000011920929',
  '-0.4',
  '-0.400000005960464',
  '-0.5',
  '-0.6',
  '-0.600000023841858',
  '-0.699999988079071',
  '-0.7',


In [13]:
test = pd.read_csv("./test_kor.csv", encoding='ms949')
test_col = test.columns

In [14]:
test = test.drop(['발생지시도','발생지시군구'], 1)

In [15]:
test_array = np.array(test)
new_dataset = []

for i in range(len(test_array)):
        new_dataset.append(test_array[i])
        
new_dataset = np.array(new_dataset)

In [16]:
str_col2 = []
for i, w in enumerate(new_dataset[0]):
    if type(w) is str:
        str_col2.append(i)

In [17]:
for idx in range(len(new_dataset)):
    for i, col in enumerate(str_col2):
        new_dataset[idx,col] = int(uniq_class[i].index(new_dataset[idx,col]))
        
# Save processed Dataset as csv file with one-hot encoding
new_dataset_pd = pd.DataFrame(new_dataset, columns=test_col)

ValueError: nan is not in list

In [ ]:
new_dataset_pd.to_csv("./wt.csv",index=None,encoding='ms949')